# Using OctoAI SVD on SageMaker through Model Packages

OctoAI SVD is a performant and feature-rich SVD implementation, allowing users to easily generate animated images. You can read more about what's possible in this [documentation](https://octo.ai/docs/octostack/sagemaker)

This sample notebook shows you how to deploy OctoAI SVD using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you recieved this notebook from model catalog on SageMaker AWS Management Console.

Note that you will have to subscribe to the OctoAI SDXL product on the AWS Marketplace.

In [41]:
# Mapping for Model Packages (initially only us-east-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/octoai-svd-5cb5fd82f53424e55fd-269e1b5374bd3ee296b64458f8222d1f"
}

In [11]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [27]:
boto_session = boto3.Session()
region = boto_session.region_name.strip()
if region not in model_package_map.keys():
    raise ValueError("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [28]:

sagemaker_session = sage.Session(boto_session=boto_session)
role = get_execution_role(sagemaker_session=sagemaker_session)

runtime_sm_client = boto_session.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [42]:
# NOTE: you can change this to whatever you'd like to name your SVD endpoint
model_name = "svd"

initial_instance_count = 1

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
    # other possible options are:
    # ml.g5.xlarge
    # ml.g5.2xlarge
    # ml.g5.4xlarge
    # ml.g5.8xlarge
    # ml.g5.12xlarge
    # ml.g5.16xlarge
    # ml.g5.24xlarge
    # ml.g5.48xlarge
    # ml.p4d.24xlarge
    # ml.p4de.24xlarge
    # ml.p5.48xlarge
)

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(initial_instance_count, real_time_inference_instance_type, endpoint_name=model_name, model_data_download_timeout=1200, container_startup_health_check_timeout=600)

Once endpoint has been created, you can use it to generate images, where you can see examples in the next section.

### B. Create input payload

You can use the SVD API to generate animated images. You can look to the [documentation](https://octo.ai/docs/octostack/sagemaker) to see a complete description.

In [44]:
# A simple payload, generating a single animated image
import os
from base64 import b64encode, b64decode

# Read the image and encode it as base64
init_image = b64encode(open('./rocket.png', 'rb').read()).decode("utf-8")

payload = {
    "cfg_scale": 1.2,
    "num_videos": 1,
    "motion_scale": 0.5,
    "noise_aug_strength": 0.01,
    "steps": 25,
    "seed": 888,
    "image": init_image,
}

response = runtime_sm_client.invoke_endpoint(
    EndpointName=model_name,
    ContentType=content_type,
    Body=json.dumps(payload),
)

output = json.loads(response["Body"].read().decode("utf8"))
print("Generated video")

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

In [45]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=model_name,
    ContentType=content_type,
    Body=json.dumps(payload),
)

output = json.loads(response["Body"].read().decode("utf8"))

### D. Visualize output

OctoAI SVD will return a JSON payload that includes all the animations generated, each encoded in base64. The structure looks like this:

```
    {
        "videos": [
            {
                "video":"<base64 encoded video>",
                "removed_for_safety":<true|false>,
                "seed":<integer>
                "safety_score":<float>
            },
            { ... }
        ],
        "prediction_time_ms":<float>
    }
```

First, let's define a simple helper function that will take the output and display all the generated animated images (in the case where more than a single animated image was returned)

In [33]:
from IPython.display import Video 
from base64 import b64decode

def display_output(endpoint_output):
    # Convert each base64-encoded animated image and display all of them
    vids = []
    for output_vid in endpoint_output["videos"]:
        decoded_video = b64decode(output_vid["video"])
        vids.append(display.Video(decoded_video))
    display.display(*vids)

We can now use it on the output we previously received

In [46]:
display_output(output)

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [50]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Clean-up

### A. Delete the model

In [51]:
model.delete_model()